In [ ]:
import numpy as np
import pandas as pd
import itertools
import h5py
import matplotlib.pyplot as plt
from scipy import stats
from scipy import special
from scipy import integrate
from scipy import interpolate
from scipy import linalg
from scipy import signal
from scipy.optimize import curve_fit
import time
from pathlib import Path
import os
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['text.usetex'] = True

import sys
import mpld3

#mpld3.enable_notebook()

sys.path.insert(0, '..')
from modules import structure_func as sf
from modules import regression as rg

In [ ]:
file_path = Path('../data/cosmograil_data/HE0435_Bonvin2016.rdb_.txt')

qso_id = file_path.name.split('_')[0]
data = pd.read_table(file_path)

t = data['mhjd'].to_numpy(dtype=np.float64)
A = data['mag_A'].to_numpy(dtype=np.float64)
errA = data['magerr_A'].to_numpy(dtype=np.float64)

N = len(t)

In [ ]:
smoothing_val = 1e-1

qso_spline = interpolate.UnivariateSpline(t, A, s=smoothing_val, k=3)
abs_diff = np.abs((qso_spline(t)-A)/A.mean())
frac_err_max = abs_diff.max()
print(frac_err_max)

new_t = np.arange(t[0], t[-1], step=0.2)

plt.scatter(t, A)
plt.plot(new_t, qso_spline(new_t), color='r')

In [ ]:
mean_square = integrate.simps(qso_spline(new_t)**2, new_t)/(t[-1]-t[0])
squared_mean = (integrate.simps(qso_spline(new_t), new_t)/(t[-1]-t[0]))**2

print(mean_square)
print(squared_mean)

In [ ]:
tau, v = sf.estimate_structure_func_from_data(t, A, errA)

In [ ]:
def power_law_sf(tau, slope, intercept):
    return 10**intercept * tau**slope

def exp_sf(tau, V0, dt0):
    return V0*(1-np.exp(-tau/dt0))

def fit_sf(tau, v, cut_off):
    pars = stats.linregress(np.log10(tau[:cut_off]), np.log10(v[:cut_off]))
    slope = pars.slope
    intercept = pars.intercept
    return slope, intercept

def spline_sf(tau, v):
    spline = interpolate.UnivariateSpline(tau, v, s=1e-6, k=3)
    return spline

In [ ]:
cut_off = int(0.75*len(tau))
slope, intercept = fit_sf(tau, v, cut_off)
popt, pcov = curve_fit(exp_sf, tau[:cut_off], v[:cut_off])

plt.loglog(tau, v, linestyle='None', marker='o', color='g', label='SF')
plt.loglog(tau[:cut_off], v[:cut_off], linestyle='None', marker='o', color='b', label='SF with cutoff')
plt.loglog(tau[:cut_off], power_law_sf(tau[:cut_off], slope, intercept), color='r', 
           label=f'power law: slope {slope:.3f} intercept {intercept:.3f}')
plt.loglog(tau, exp_sf(tau, *popt), color='orange', 
           label=f'exponential: V0 {popt[0]:.3f} dt0 {popt[1]:.3f}')
# v_spline = spline_sf(tau, v)
# plt.loglog(tau, v_spline(tau))
plt.legend(fontsize=12)
print('Max lag considered: %s days ' % tau[cut_off-1])
print('Max existing lag: %s days ' % tau.max())

In [ ]:
delay=50

t_doubled = np.concatenate([t, t - delay])
tau_doubled = sf.compute_lags_matrix(t_doubled)
V_matrix = power_law_sf(tau_doubled, slope, intercept)
s2 = mean_square
C  = s2 - V_matrix
C += np.eye(2*N) * (np.concatenate([errA, errA]))**2
L  = np.linalg.cholesky(C)

In [ ]:
r = np.random.normal(0, 1, size=2*N)
y = L @ r

yA = y[:N]
yB = y[N:]

plt.scatter(t, yA)
plt.scatter(t, yB)

In [ ]:
idx=9
plt.scatter(t, curves[idx][0] - curves[idx][0].mean(), label='realization %s' % idx, color='r')
plt.scatter(t, A-A.mean(), color='black')

In [ ]:
sim_sfs = []

for idx in range(10):
    _, v_sim = sf.estimate_structure_func_from_data(t, curves[idx][0], errA)
    sim_sfs.append(v_sim)

In [ ]:
plt.loglog(tau, sim_sfs[idx], linestyle='None', marker='o', color='r')
plt.loglog(tau, v, linestyle='None', marker='o', color='black')

In [ ]:
delays = np.arange(1, 100, 1)

curves = []

for delay in delays:
    N = len(t)
    t_doubled = np.concatenate([t, t - delay])
    tau_doubled = sf.compute_lags_matrix(t_doubled)
    A2_mean = (A**2).mean()
    C = A2_mean - exp_sf(tau_doubled, *popt)
    # C = A2_mean - power_law_sf(tau_doubled, slope, intercept)
    # C += 1e-10 * np.eye(2*N)
    errA_doubled = np.concatenate([errA, errA])
    L = np.linalg.cholesky(C)
    y = L @ np.random.normal(0, 1, size=2*N) + errA_doubled**2 * np.random.normal(0, 1, size=2*N)
    curves.append([y[:N], y[N:]])
"""
L = np.linalg.cholesky(C)
y = L @ np.random.normal(0, 1, size=2*N) + errA_doubled**2 * np.random.normal(0, 1, size=2*N)
"""